# Build RNN to generate hunger games sounding text

based on tutorial in here : https://www.tensorflow.org/text/tutorials/text_generation

Could not use all features since I have tf version 2.5

In [1]:
import sys, os
import tensorflow as tf
import pandas as pd
import numpy as np

Init Plugin
Init Graph Optimizer
Init Kernel


In [109]:
tf.__version__

'2.5.0'

In [5]:
with open("(1) The Hunger Games.txt", 'rb') as file : 
    txt_file = file.readlines()
type(txt_file)

list

In [11]:
len(txt_file)

1071

In [25]:
txt_all = [str(x)[2:] for x in txt_file][1:]
txt_all_long = [x.replace('\\r','\\s').replace('\\n','\\s') 
                for x in txt_all if len(x) > 49]

487

In [34]:
single_text = ' '.join(txt_all_long)
vocab = sorted(set(single_text))
len(vocab)

74

In [36]:
in_size, out_size = 200, 100
def split_input_sequence(seq) : 
    """
    split sequence into all but last & all but first
    """
    input_text = seq[:-1]
    output_text = seq[1:]
    return input_text, output_text


In [43]:
uc_split = tf.strings.unicode_split(single_text, input_encoding='UTF-8')
data = tf.data.Dataset.from_tensor_slices(uc_split)

In [88]:
vocab_lookup = {str(x):i for i,x in enumerate(list(vocab))}
letter_lookup = {i:str(x) for i,x in enumerate(list(vocab))}
uc_np = pd.Series(uc_split.numpy().astype(str)).astype(str).replace(vocab_lookup).values
tens = tf.convert_to_tensor(uc_np, dtype=tf.int64)

In [89]:
ids_dataset = tf.data.Dataset.from_tensor_slices(tens)

In [90]:
def translate(n) : 
    """
    translate
    """
    return letter_lookup[n]
for ids in ids_dataset.take(10):
    print(translate(ids.numpy()))

W
h
e
n
 
I
 
w
a
k


In [98]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1) : 
    print(seq)

tf.Tensor(
[44 55 52 61  0 30  0 70 48 58 52  0 68 63  5  0 67 55 52  0 62 67 55 52
 65  0 66 56 51 52  0 62 53  0 67 55 52  0 49 52 51  0 56 66  0 50 62 59
 51  7  0 34 72  0 53 56 61 54 52 65 66  0 66 67 65 52 67 50 55  0 62 68
 67  5  0 66 52 52 58 56 61 54  0 37 65 56 60 66  0 70 48 65 60 67 55  0
 49 68 67  0 53], shape=(101,), dtype=int64)


In [102]:
dataset = sequences.map(split_input_sequence)

for input_example, target_example in dataset.take(1):
    print("Input :", (input_example).numpy())
    print("Target:", (target_example).numpy())

Input : [44 55 52 61  0 30  0 70 48 58 52  0 68 63  5  0 67 55 52  0 62 67 55 52
 65  0 66 56 51 52  0 62 53  0 67 55 52  0 49 52 51  0 56 66  0 50 62 59
 51  7  0 34 72  0 53 56 61 54 52 65 66  0 66 67 65 52 67 50 55  0 62 68
 67  5  0 66 52 52 58 56 61 54  0 37 65 56 60 66  0 70 48 65 60 67 55  0
 49 68 67  0]
Target: [55 52 61  0 30  0 70 48 58 52  0 68 63  5  0 67 55 52  0 62 67 55 52 65
  0 66 56 51 52  0 62 53  0 67 55 52  0 49 52 51  0 56 66  0 50 62 59 51
  7  0 34 72  0 53 56 61 54 52 65 66  0 66 67 65 52 67 50 55  0 62 68 67
  5  0 66 52 52 58 56 61 54  0 37 65 56 60 66  0 70 48 65 60 67 55  0 49
 68 67  0 53]


2022-07-05 13:29:39.019665: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-05 13:29:39.021647: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [103]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [110]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 256

class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [112]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [113]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 74) # (batch_size, sequence_length, vocab_size)


In [114]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  9472      
_________________________________________________________________
gru_2 (GRU)                  multiple                  296448    
_________________________________________________________________
dense_2 (Dense)              multiple                  19018     
Total params: 324,938
Trainable params: 324,938
Non-trainable params: 0
_________________________________________________________________


### Untrained model prediction

In [116]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([65,  5, 14, 32, 42, 21, 14, 24, 50, 32, 34,  0, 35,  4, 57, 37, 65,
       70, 36, 66, 49, 70, 13,  6, 35, 20, 48, 24, 69, 54, 15, 60,  5, 71,
        9, 43, 34, 64, 70, 24, 49, 33, 64, 62,  8, 60, 20, 48, 11, 44, 25,
        5,  4,  7, 45, 70,  4, 26, 32,  2, 49, 61, 27, 65, 27,  1, 67, 23,
       59, 55, 35, 73,  9, 32, 62, 63,  3, 68,  5, 38, 65,  1, 65, 20, 36,
       68, 31, 29, 73, 31, 14, 46, 15, 38, 66,  0, 15,  3, 31, 57])

In [123]:
def translate_array(arr) : 
    """
    translate over & over on an array
    """
    return np.array([translate(a) for a in arr])
def translate_array_join(arr) : 
    """
    join the text
    """
    return ''.join(translate_array(arr))

In [124]:
print("Input:\n", translate_array_join(input_example_batch[0].numpy()))
print()
print("Next Char Predictions:\n", translate_array_join(sampled_indices))

Input:
  jackets another matter. Stinking and scorched, at least\s\s' a foot of the back beyond repair. I cu

Next Char Predictions:
 r,5KU?5CcKM N(jPrwOsbw4-N;aCvg6m,x0VMqwCbLqo/m;a2WD,(.Xw(EK"bnFrF!tBlhNz0Kop'u,Qr!r;OuJHzJ5Y6Qs 6'Jj


### Train model

In [127]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)
EPOCHS = 20
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


2022-07-05 13:40:15.522739: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-07-05 13:40:15.878661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-07-05 13:40:17.370911: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 [==============================] - 10s 60ms/step - loss: 3.2342
Epoch 2/20
80/80 [==============================] - 5s 61ms/step - loss: 2.8666
Epoch 3/20
80/80 [==============================] - 5s 60ms/step - loss: 2.7125
Epoch 4/20
80/80 [==============================] - 5s 60ms/step - loss: 2.7631
Epoch 5/20
80/80 [==============================] - 5s 60ms/step - loss: 2.7692
Epoch 6/20
80/80 [==============================] - 5s 60ms/step - loss: 2.6542
Epoch 7/20
80/80 [==============================] - 5s 58ms/step - loss: 2.5444
Epoch 8/20
80/80 [==============================] - 5s 59ms/step - loss: 2.4711
Epoch 9/20
80/80 [==============================] - 5s 59ms/step - loss: 2.3555
Epoch 10/20
80/80 [==============================] - 5s 59ms/step - loss: 2.2355
Epoch 11/20
80/80 [==============================] - 5s 59ms/step - loss: 2.1279
Epoch 12/20
80/80 [==============================] - 5s 59ms/step - loss: 2.0696
Epoch 13/20
80/80 [============================

In [143]:
x,y = model(inputs=tf.convert_to_tensor([[0,2,1]*30,[1,2]*45],dtype=tf.int64),states=None,return_state=True)
x.shape, y.shape

(TensorShape([2, 90, 74]), TensorShape([2, 256]))